In [1]:
import tensorflow as tf
import pandas as pd
import glob
import json
import numpy as np
import random
import os
import tensorflow_io as tfio

In [2]:
def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_it_all()

In [3]:
class CFG:
    left_ROWS_per_frame = 21
    sequence_length = 20
    batch_size = 32

labels  = json.load(open('sign_to_prediction_index_map.json','r'))
complete_df = pd.read_csv('train.csv')
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(complete_df, test_size=0.2)

In [4]:
def build_loader(with_labels=True):
    def load_video(video_path):
        #print('herer')
        video_df = tfio.IODataset.from_parquet(video_path)
        #video_df = pd.read_parquet(video_path, engine='pyarrow')
        #video_df.fillna(0,inplace=True)
        left_df = video_df[video_df.type=='left_hand']
        left_values = left_df[['x','y','z']].values
        left_values = left_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        left_hand_array =  tf.image.resize(left_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        right_df = video_df[video_df.type=='right_hand']
        right_values = right_df[['x','y','z']].values
        right_values = right_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        right_hand_array =  tf.image.resize(right_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        return [left_hand_array, right_hand_array]
    
    def load_video_with_labels(path, label):
        return load_video(path), labels[label]
    
    return load_video_with_labels if with_labels else load_video

In [5]:
class CustomData(tf.keras.utils.Sequence):
    def __init__(self,df,num_frames=20,batch_size=8,shuffle=True,\
                 labels_path='sign_to_prediction_index_map.json'):
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_frames = num_frames
        self.labels  = json.load(open('sign_to_prediction_index_map.json','r'))
        self.on_epoch_end()
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self,index):
        batches = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        left_hand_input = np.zeros(shape=(self.batch_size,self.num_frames,CFG.left_ROWS_per_frame,3))
        right_hand_input = np.zeros(shape=(self.batch_size,self.num_frames,CFG.left_ROWS_per_frame,3))
        labels = []
        for i,row_val in enumerate(batches):
            row = self.df.iloc[row_val]
            left_hand,right_hand = self.load_video(row['path'])
            left_hand_input[i,:] = left_hand
            right_hand_input[i,:] = right_hand
            labels.append(self.labels[row['sign']])
        return [left_hand_input,right_hand_input],np.asarray(labels)
            
    def load_video(self,video_path):
        video_df = pd.read_parquet(video_path, engine='pyarrow')
        video_df.dropna(inplace=True)
        left_df = video_df[video_df.type=='left_hand']
        left_values = left_df[['x','y','z']].values
        left_values = left_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        if len(left_values)!=0:
            left_hand_array =  tf.image.resize(left_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        else:
            left_hand_array =  tf.zeros(shape=(CFG.sequence_length, CFG.left_ROWS_per_frame,3),dtype=tf.float32)
        
        right_df = video_df[video_df.type=='right_hand']
        right_values = right_df[['x','y','z']].values
        right_values = right_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        if len(right_values) != 0:
            right_hand_array =  tf.image.resize(right_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        else:
            right_hand_array =  tf.zeros(shape=(CFG.sequence_length, CFG.left_ROWS_per_frame,3),dtype=tf.float32)
        return left_hand_array, right_hand_array
    
    def __len__(self):
        return len(self.df)//self.batch_size

In [6]:
train_datagen = CustomData(train_df,num_frames=CFG.sequence_length,batch_size=CFG.batch_size)
test_datagen = CustomData(test_df,num_frames=CFG.sequence_length,batch_size=CFG.batch_size)

In [7]:
def conv1d_lstm_block(inputs, filters):
    vector = tf.keras.layers.ConvLSTM1D(filters=32, kernel_size=5)(inputs)
    vector = tf.keras.layers.Dropout(0.1)(vector)
    vector = tf.keras.layers.ConvLSTM1D(filters=64, kernel_size=5)(inputs)
    vector = tf.keras.layers.Dropout(0.1)(vector)
    return vector

def get_model():
    input1 = tf.keras.Input((CFG.sequence_length, CFG.left_ROWS_per_frame, 3), dtype=tf.float32)
    input2 = tf.keras.Input((CFG.sequence_length, CFG.left_ROWS_per_frame, 3), dtype=tf.float32)
    left_hand_vector = conv1d_lstm_block(input1, [64])
    right_hand_vector = conv1d_lstm_block(input2, [64])
    vector = tf.keras.layers.Concatenate(axis=1)([left_hand_vector, right_hand_vector])
    vector = tf.keras.layers.Flatten()(vector)
    output = tf.keras.layers.Dense(250, activation="softmax")(vector)
    model = tf.keras.Model(inputs=[input1,input2], outputs=output)
    model.compile(tf.keras.optimizers.Adam(0.000333), "sparse_categorical_crossentropy", metrics="accuracy")
    return model

In [8]:
model = get_model()

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 21, 3)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 20, 21, 3)]  0           []                               
                                                                                                  
 conv_lstm1d_1 (ConvLSTM1D)     (None, 17, 64)       86016       ['input_1[0][0]']                
                                                                                                  
 conv_lstm1d_3 (ConvLSTM1D)     (None, 17, 64)       86016       ['input_2[0][0]']                
                                                                                              

In [10]:
file_name = "models/030423_12_50.h5"
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        file_name, 
        save_best_only=True, 
        monitor="val_accuracy",
        mode="max",
        verbose = 1
    ),
    tf.keras.callbacks.EarlyStopping(
        patience=5, 
        monitor="val_accuracy",
        mode="max"
    ),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,mode='max',verbose=1,
                              patience=5, min_lr=0.000001)
]
model.fit(train_datagen,validation_data=test_datagen,\
          epochs=30, callbacks=callbacks)
model = tf.keras.models.load_model(file_name)

Epoch 1/30
2361/2361 [==============================] - ETA: 0s - loss: 3.7721 - accuracy: 0.1959
Epoch 1: val_accuracy improved from -inf to 0.35037, saving model to models/030423_12_50.h5
2361/2361 [==============================] - 1129s 476ms/step - loss: 3.7721 - accuracy: 0.1959 - val_loss: 2.8099 - val_accuracy: 0.3504 - lr: 3.3300e-04
Epoch 2/30
2361/2361 [==============================] - ETA: 0s - loss: 2.5196 - accuracy: 0.4004
Epoch 2: val_accuracy improved from 0.35037 to 0.44354, saving model to models/030423_12_50.h5
2361/2361 [==============================] - 1114s 472ms/step - loss: 2.5196 - accuracy: 0.4004 - val_loss: 2.3328 - val_accuracy: 0.4435 - lr: 3.3300e-04
Epoch 3/30
2361/2361 [==============================] - ETA: 0s - loss: 2.1190 - accuracy: 0.4825
Epoch 3: val_accuracy improved from 0.44354 to 0.49163, saving model to models/030423_12_50.h5
2361/2361 [==============================] - 1111s 471ms/step - loss: 2.1190 - accuracy: 0.4825 - val_loss: 2.1059

2361/2361 [==============================] - 1139s 482ms/step - loss: 0.4217 - accuracy: 0.8784 - val_loss: 1.9055 - val_accuracy: 0.6013 - lr: 3.3300e-04
Epoch 26/30
2361/2361 [==============================] - ETA: 0s - loss: 0.3921 - accuracy: 0.8871
Epoch 26: val_accuracy did not improve from 0.60365
2361/2361 [==============================] - 1126s 477ms/step - loss: 0.3921 - accuracy: 0.8871 - val_loss: 1.9700 - val_accuracy: 0.5938 - lr: 3.3300e-04
Epoch 27/30
2361/2361 [==============================] - ETA: 0s - loss: 0.3703 - accuracy: 0.8939
Epoch 27: val_accuracy did not improve from 0.60365
2361/2361 [==============================] - 1128s 478ms/step - loss: 0.3703 - accuracy: 0.8939 - val_loss: 1.9767 - val_accuracy: 0.5942 - lr: 3.3300e-04
Epoch 28/30
2361/2361 [==============================] - ETA: 0s - loss: 0.3521 - accuracy: 0.9000
Epoch 28: val_accuracy did not improve from 0.60365
2361/2361 [==============================] - 1128s 478ms/step - loss: 0.3521 - acc